In [1]:
import json

In [2]:

# Read the JSON data from the file
with open('piazza_di_popolo.plan', 'r') as file:
    plan_data = json.load(file)

# Print the dictionary
print(plan_data)

{'fileType': 'Plan', 'geoFence': {'circles': [], 'polygons': [], 'version': 2}, 'groundStation': 'QGroundControl', 'mission': {'cruiseSpeed': 15, 'firmwareType': 12, 'globalPlanAltitudeMode': 1, 'hoverSpeed': 5, 'items': [{'autoContinue': True, 'command': 530, 'doJumpId': 1, 'frame': 2, 'params': [0, 2, None, None, None, None, None], 'type': 'SimpleItem'}, {'AMSLAltAboveTerrain': None, 'Altitude': 50, 'AltitudeMode': 1, 'autoContinue': True, 'command': 22, 'doJumpId': 2, 'frame': 3, 'params': [0, 0, 0, None, 41.912876711789345, 12.474100697522232, 50], 'type': 'SimpleItem'}, {'TransectStyleComplexItem': {'CameraCalc': {'AdjustedFootprintFrontal': 24.41463414634146, 'AdjustedFootprintSide': 32.49999999999999, 'CameraName': 'Custom Camera', 'DistanceMode': 1, 'DistanceToSurface': 50, 'FixedOrientation': False, 'FocalLength': 3, 'FrontalOverlap': 70, 'ImageDensity': 3.3028455284552845, 'ImageHeight': 2464, 'ImageWidth': 3280, 'Landscape': True, 'MinTriggerInterval': 0, 'SensorHeight': 6.5

In [3]:
home_position = plan_data['mission']['plannedHomePosition']

In [4]:

n_items = len(plan_data['mission']['items'])
waypoints = []
image_capture_waypoints = []

for i in range(n_items):
    item = plan_data['mission']['items'][i]
    if 'command' in item:
        if item['command']==530:
            print("arming")
        if item['command']==22:
            print('takeoff to {} m '.format(item['params'][-1]) )
    elif 'TransectStyleComplexItem' in item: #survey or corridor scan
        if item['complexItemType']=='survey':
            print("survey")
            survey = item['TransectStyleComplexItem']
            for n, waypoint_item in enumerate(survey['Items']):
                
                if waypoint_item['command']==16: #MAV_CMD_NAV_WAYPOINT
                    waypoints.append(waypoint_item['params'][4:7])
                elif waypoint_item['command']==2000: #MAV_CMD_IMAGE_START_CAPTURE
                    image_capture_waypoints.append(waypoints[-1])
                else:
                    print(waypoint_item)
            assert survey['CameraShots']==len(image_capture_waypoints)
    else:
        print(item)
    # command 2000 is MAV_CMD_IMAGE_START_CAPTURE 
    # 2001 MAV_CMD_IMAGE_STOP_CAPTURE
    # command 16 is MAV_CMD_NAV_WAYPOINT
    
    # https://mavlink.io/en/messages/common.html

arming
takeoff to 50 m 
survey


In [33]:
# plan_data['mission']

In [17]:
# plan_data['mission']['items']

In [5]:
!touch waypoints.txt

In [ ]:
with open('waypoints.txt', 'w') as f:
    for w in waypoints:
        f.write(f"{w}\n")